# Logistic Regression Example using Blood Transfusion Service Center  Data Set
## For 2017-03-11 Data Science with Python Study Group
#### Dinesh Shenoy 

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',1000)

from matplotlib import pyplot as plt
%matplotlib inline

random_state = 42

### 1.  Pick a data set.   UCI Machine Learning Library:  https://archive.ics.uci.edu/ml/datasets.html
* See left-hand frame options for refining your search.
* For this demo I searched for one with just a few features / predictors / attributes and a simple yes/no outcome to be classified:  
    * **Blood Transfusion Service Center Data Set**:  https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center
* Other ones that looked interesting to me as potentially good examples for applying Logistic Regression to are:
    * **Mammographic Mass Data Set**:  https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass
    * **Banknote Authentication Data Set**:  https://archive.ics.uci.edu/ml/datasets/banknote+authentication#
    * **Haberman's Survival Data Set**:  https://archive.ics.uci.edu/ml/datasets/Haberman%27s+Survival

### 2.  Blood Transfusion Service Center Data Set:  https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center

#### The predictors are:
    * R = Recency = months since last donation
    * F = Frequency = total number of donations
    * M = Monetary = total blood donated in c.c. . . . huh???
    * T = Time = months since first donation

#### The response is:
    * D = a binary variable representing whether he/she donated blood in March 2007: 0 = did not, 1 = did

In [ ]:
# examine the data file before importing
!head transfusion.data.txt

In [ ]:
# read data into a Pandas dataframe-- does Pandas figure out the headers?
dat = pd.read_csv('transfusion.data.txt')

print(dat.shape)
dat.head()

In [ ]:
# re-do, with shorter names for the columns and skip that first row
cols = ['Recency','Frequency','Monetary','Time','Donated']

dat = pd.read_csv('transfusion.data.txt',names=cols,skiprows=1)

print(dat.shape)
dat.head()

### 3.  Take a quick look at the predictors, any issues?

In [ ]:
plt.rcParams['figure.figsize'] = (18,10)
dat.plot(subplots=True)
plt.show()

### Things I note:
    * The data might have been sorted by Recency, in two batches . . . not a big deal, if we split randomly.
    * Frequency and Monetary appear suspiciously correlated.
    * Donated = 0 seems much more common than Donated = 1.

In [ ]:
# count the number of 0's and 1's in the response column "Donated"
np.bincount(dat.Donated)

In [ ]:
# check the data types
print(dat.dtypes)

In [ ]:
# convert the predictors to floats (maybe not necessary, but . . . )
dat[dat.columns[:-1]] *= 1.0
print(dat.dtypes)

### 4.  Double-check if any missing values (the notes on the data say no missing values, but . . . )

In [ ]:
dat.isnull().values.any()

### 5.  Slice the frame into a DataFrame for the predictors and a Series for the response

In [ ]:
predictors = dat[cols[:-1]].copy()   
response   = dat[cols[ -1]].copy()   

In [ ]:
predictors.head()

In [ ]:
response.head()

### 6. Check for correlations amongst the predictors

In [ ]:
predictors.corr()

In [ ]:
# visualize the correlation using pyplot.matshow() to display pandas.DataFrame.corr()
def plot_corr(df, size=11):
    """
    Plots correlation matrix for each pair of columns    
    """
    corr = df.corr()
    fig, ax = plt.subplots(figsize = (size,size))
    cax = ax.matshow(corr)
    fig.colorbar(cax, fraction=0.0458, pad=0.04)
    plt.xticks(np.arange(len(corr.columns)), corr.columns)
    plt.yticks(np.arange(len(corr.columns)), corr.columns)

plot_corr(predictors, size=5)

In [ ]:
# yeah, clearly one column was scaled from the other
set( predictors.Monetary.values / predictors.Frequency.values )

In [ ]:
# drop the "Monetary" column
del predictors['Monetary']
predictors.head()

### 7. Normalize the predictors:  http://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
plt.rcParams['figure.figsize'] = (16,8)
predictors.hist()
plt.show()

In [ ]:
from sklearn import preprocessing
predictors = preprocessing.scale(predictors)

# package the resulting array back into a DataFrame
predictors = pd.DataFrame(data=predictors,index=np.arange(0,predictors.shape[0]),columns=['Recency','Frequency','Time'])

plt.rcParams['figure.figsize'] = (16,8)
predictors.hist()
plt.show()

In [ ]:
# mean of each predictor is 0, std deviation is 1
predictors.describe()

### 8.  Split the data into "train" and "test" sets
#### Use `sklearn.model_selection.train_test_split()` to randomize the split.
#### Use "`X`" for the matrix of predictors and "`y`" for the vector of responses, following the convention in the sklearn docs at http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

test_split_frac = 0.3

X_train, X_test, y_train, y_test = train_test_split(predictors, response, 
                                                            test_size=test_split_frac, random_state=random_state)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# To confirm the split was randomized, note the row indices
X_train.head()

In [ ]:
# confirm they match up in the response (no scrambling of indices!)
y_train.head()

In [ ]:
# for comparison with the confusion matrix below, count how many actual "0" and "1" responses
# in the test data set
n_true_positive = y_test.sum()
n_true_negative = y_test.shape[0] - y_test.sum()
n_true_positive, n_true_negative

### 9. Train a LogisticRegression classifier on `X_train` and `y_train`

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()  # keep all the defaults 

lr_model.fit(X_train,y_train)

### 10. Test the model on `X_test` and `y_test`

In [ ]:
lr_predict_test = lr_model.predict(X_test)
lr_predict_test.shape

In [ ]:
# how many incorrectly predicted response values out of 225?
len(np.nonzero( y_test.values - lr_predict_test )[0])

### 11. Confusion matrix
`sklearn.metrics.confusion_matrix()` generates a confusion matrix. Supplied with option `labels=[1,0]`, it produces a confusion matrix in which:

![title](Binary_Classification_Matrix_Definition.png)
(Image credit:  https://docs.wso2.com/display/ML100/Model+Evaluation+Measures)

A *perfect* classifier's confusion matrix for this test data set should be:

$$\begin{bmatrix} 60 & 0 \\ 0 & 165 \end{bmatrix}$$

Compare to how we actually did . . . 

In [ ]:
from sklearn import metrics

# "labels" is a list of the response values: 1 or 0; put 1 first in the list to match layout shown above
response_labels = [1,0]

metrics.confusion_matrix(y_test, lr_predict_test, labels=response_labels)

### 12.  Recall ("Sensitivity") and Precision
#### *Recall* (or "sensitivity") is how well the model is correctly predicting a "1" (did donate) when the person in fact did donate.  That is the number of true positives (TP) divided by the total actual positive (TP + FN == all that actually donated):  

$$\mbox{Recall} = \frac{TP}{TP+FN}$$

Increasing recall would come through decreasing false negatives (FN).

#### *Precision* is how often a person actually did donate when the model *said* it they donated.  That is the number of true positives (TP) divided by the total number of people the model said donated (TP + FP == all the people the model *said* donated):

$$\mbox{Precision} = \frac{TP}{TP+FP}$$

Increasing precision would come through decreasing false positives (FP).

In [ ]:
# get recall (and precision) as part of the classification report
print(metrics.classification_report(y_test, lr_predict_test, labels=response_labels))

#### For the response class of "0", the recall is 98%, which sounds almost too good . .  .
#### More importantly, for response class of "1", the recall is 8%.  We'd be better off flipping a coin!
#### Re-examine the imbalance of "0" and "1" noted earlier in the data overall and in the training data specifically:

In [ ]:
print("all data fraction of 1's = %.3f" % (      1. * dat.Donated.values.sum() / dat.Donated.values.shape[0] ) )
print("all data fraction of 0's = %.3f" % ( 1. - 1. * dat.Donated.values.sum() / dat.Donated.values.shape[0] ) )

In [ ]:
print("y_train fraction of 1's = %.3f" % (      1. * y_train.sum() / y_train.shape[0] ) )
print("y_train fraction of 0's = %.3f" % ( 1. - 1. * y_train.sum() / y_train.shape[0] ) )

In [ ]:
# and here's the raw count of 0's and 1's in the training data set
np.bincount(y_train)

### 13.  Unbalanced classes -- re-run with `LogisticRegression( class_weight='balanced' )`
From the docs at http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression:

The “balanced” mode uses the values of y to automatically adjust weights inversely  proportional to class frequencies in the input data

In [ ]:
lr_model = LogisticRegression(class_weight='balanced')  

lr_model.fit(X_train,y_train)
lr_predict_test = lr_model.predict(X_test)

metrics.confusion_matrix(y_test, lr_predict_test, labels=response_labels)

In [ ]:
print(metrics.classification_report(y_test, lr_predict_test, labels=response_labels))

### 13. OPTIONAL:  Cross Validation
    * Overfitting is a problem (in general, if not here)
    * Cutting up the data into k-folds, training on the k-1 and testing on the kth should yield a more stable model

![title](k-fold-cross-validation.jpg)
(Image credit:  http://cse3521.artifice.cc/classification-evaluation.html)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

# cv = 10 means make 10-folds within the Training set
# Cs = 3 means within each fold, make 3 attempts to find best tuning parameter
lr_model_CV = LogisticRegressionCV(Cs=3, cv=10, refit=True, class_weight='balanced')

lr_model_CV.fit(X_train, y_train)

In [ ]:
lr_CV_predict = lr_model_CV.predict(X_test)

metrics.confusion_matrix(y_test, lr_CV_predict, labels=response_labels)

In [ ]:
print(metrics.classification_report(y_test, lr_CV_predict, labels=response_labels))